<a href="https://colab.research.google.com/github/Shihiro-Sasaki/AnimalVocalizationAnalysis/blob/main/AnimalVocalizationAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ライブラリをimport
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd
import shutil
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive') #Google Drive上のデータを使用する場合のみ必要
# --- ユーザー設定エリア ---
%cd #ディレクトリを指定
# -----------------------

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SL/Python


In [ ]:
def analyze_animal_vocalization(file_path,output_name):
  # --- ユーザー設定エリア ---
  spectrogram_output_dir='spectrogram'  #アウトプットディレクトリをそれぞれ指定
  centroid_output_dir='centroid'
  # -----------------------

  #音声データを読み込み
  y, sr=librosa.load(file_path, sr=None)

  #ナイキスト周波数に合わせた8000Hz、倍音構造の観察のための1500Hz、1000Hzを軸の上限としたスペクトログラムで可視化
  for i in (8000,1500,1000):
    FIXED_SR = 16000
    FREQ_MAX = i
    DB_MIN = -80
    DB_MAX = 0

    #stft実行
    # 低周波の微細な変化を確認するため、n_fftは2048（約46ms @44.1kHz）に設定
    stft_abs = np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
    stft_db = librosa.amplitude_to_db(stft_abs, ref=np.max)

    #グラフを生成
    plt.figure(figsize=(12,6))
    img = librosa.display.specshow(stft_db, sr=sr, hop_length=512,x_axis='time', y_axis='hz',vmin=DB_MIN, vmax=DB_MAX) # 色の基準を固定
    plt.ylim(0,FREQ_MAX)
    plt.colorbar(img, format='%+2.0f dB')
    plt.ylabel('Frequency(Hz)')
    plt.xlabel('time(s)')
    if i == 8000:
      plt.title(f"Spectrogram:{output_name}")
      plt.savefig(os.path.join(spectrogram_output_dir,f"Spectrogram_{output_name}.png"))
    elif i == 1500:
      plt.title(f"Spectrogram:{output_name}_below1500Hz")
      plt.savefig(os.path.join(spectrogram_output_dir,f"Spectrogram_{output_name}_below1500Hz.png"))
    elif i == 1000:
      plt.title(f"Spectrogram:{output_name}_below1000Hz")
      plt.savefig(os.path.join(spectrogram_output_dir,f"Spectrogram_{output_name}_below1000Hz.png"))

    plt.close()

  #セントロイドを出力
  FREQ_MAX = 6000

  spectral_centroid = librosa.feature.spectral_centroid(y=y,sr=sr)[0]
  spectral_centroid_smoothed = np.convolve(spectral_centroid, np.ones(5)/5, mode='same')  #重要な音響情報の損失がなく、かつ背景雑音を効果的に抑制した分析に適した窓幅N=5で平滑化
  times= librosa.frames_to_time(np.arange(len(spectral_centroid_smoothed)),sr=sr)

  #平滑化前データをグラフとして出力
  plt.figure(figsize=(12,4))
  plt.ylim(0,FREQ_MAX)
  plt.xlim(0,10)

  plt.plot(times, spectral_centroid)
  plt.ylabel('Frequency(Hz)')
  plt.xlabel('Time(s)')
  plt.title(f'Spectral_centroid:{output_name}')
  plt.grid(True)
  plt.tight_layout()
  plt.savefig(os.path.join(centroid_output_dir,f'Spectral_centroid_{output_name}.png'))
  plt.close()

  #平滑化後データをグラフとして出力
  plt.figure(figsize=(12,4))
  plt.ylim(0,FREQ_MAX)
  plt.xlim(0,10)

  plt.plot(times, spectral_centroid_smoothed)
  plt.ylabel('Frequency(Hz)')
  plt.xlabel('Time(s)')
  plt.title(f'Spectral_centroid:{output_name}_smoothed')
  plt.grid(True)
  plt.tight_layout()
  plt.savefig(os.path.join(centroid_output_dir,f'Spectral_centroid_{output_name}_smoothed.png'))
  plt.close()


  print(f"解析終了:{output_name}")

In [ ]:
# --- ユーザー設定エリア ---
pathlist=[
  #音声ファイルをリスト形式で入力
]
output_name=[
  #出力時のデータ名・グラフタイトル名をリスト形式で入力
]
# -----------------------
for j in range(len(pathlist)):
  analyze_animal_vocalization(pathlist[j],output_name[j])

解析終了:Tiger_Threat_01
解析終了:Tiger_Threat_02
解析終了:Lion_Threat_01
解析終了:Lion_Threat_02
解析終了:Lion_Affection_01
